# Progetto di Big data in Business, economics and society
### di Beatrice Fumagalli, Nicholas Missineo, Francesco Simoncelli e Beatrice Somaschini

### Caricamento librerie

In [2]:
#! pip install numpy
#! pip install pandas
#! pip install string

In [3]:
#Librerie generiche Python
import pandas as pd
import numpy as np
import pickle #Salvare Corpus 

#librerie per pre-processing
import gensim
import string
import multiprocessing #Numero di cpu
import re
from gensim.parsing.preprocessing import preprocess_string

#Librerie per i warnings
import warnings 
warnings.filterwarnings('ignore')

from tqdm import tqdm #Vedere l'avanzamento del run

C:\Users\beatr\Anaconda3\lib\site-packages\gensim\utils.py:1197: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


### Caricamento Dataset iniziale

In [4]:
data = pd.read_csv("dataset.csv", sep = ';', encoding='latin-1')
data

,Ragione sociale,Partita IVA,Provincia,Indirizzo sede legale Regione,ultimo Bilancio Ricav delle vendite,SISTEMA PREVALENTE
0,NATUZZI SPA,3513760722,Bari,Puglia,399745720,Sistema ARREDAMENTO
1,POLTRONESOFA SPA,3613140403,Bologna,Emilia-Romagna,315292721,Sistema ARREDAMENTO
2,LUBE INDUSTRIES SRL,1504060433,Macerata,Marche,191232092,Sistema ARREDAMENTO
3,SCAVOLINI SPA,113120414,Pesaro Urbino,Marche,172895557,Sistema ARREDAMENTO
4,B&B ITALIA SPA,7122350965,Milano,Lombardia,168759481,Sistema ARREDAMENTO
5,POLTRONA FRAU SPA,5079060017,Monza e della Brianza,Lombardia,163193000,Sistema ARREDAMENTO
6,POLIFORM SPA,226920130,Como,Lombardia,161540108,Sistema ARREDAMENTO
7,VENETA CUCINE SPA,3764840264,Treviso,Veneto,147732905,Sistema ARREDAMENTO
8,IMAB GROUP SPA,111950416,Pesaro Urbino,Marche,145431089,Sistema ARREDAMENTO
9,MOLTENI E C.- SPA,694950965,Monza e della Brianza,Lombardia,145417171,Sistema ARREDAMENTO


### Pre-processing Dataset

### Rinominazione colonne per facilitarne il richiamo nel codice

In [5]:
data = data.rename(index=str, columns={"Indirizzo sede legale Regione": "regione", "SISTEMA PREVALENTE": "sistema", 
                                       "Ragione sociale": "azienda", 
                                       "Provincia": "provincia",
                                       " ultimo Bilancio Ricav delle vendite ": "fatturato",
                                       "Partita IVA": "p.iva"})

### Suddivisione aziende in piccole, medie, grandi in base al fatturato

In [6]:
#pd.to_numeric(data['fatturato']).head()
#Funzione che prende un range stabilito di valori del fatturato assegna la dimensione 'micro', 'piccola', 'media' o 'grande' 
#La funzione viene poi applicata al dataset

In [7]:
def assegna_dimensione(value):
    if(value <= 2000000): return 'micro'
    elif(value > 2000000 and value <= 10000000): return 'piccola'
    elif(value > 10000000 and value <= 50000000): return 'media'
    else: return 'grande'

data['dimensione'] = data['fatturato'].apply(assegna_dimensione)

### Overview

In [8]:
data

,azienda,p.iva,provincia,regione,fatturato,sistema,dimensione
0,NATUZZI SPA,3513760722,Bari,Puglia,399745720,Sistema ARREDAMENTO,grande
1,POLTRONESOFA SPA,3613140403,Bologna,Emilia-Romagna,315292721,Sistema ARREDAMENTO,grande
2,LUBE INDUSTRIES SRL,1504060433,Macerata,Marche,191232092,Sistema ARREDAMENTO,grande
3,SCAVOLINI SPA,113120414,Pesaro Urbino,Marche,172895557,Sistema ARREDAMENTO,grande
4,B&B ITALIA SPA,7122350965,Milano,Lombardia,168759481,Sistema ARREDAMENTO,grande
5,POLTRONA FRAU SPA,5079060017,Monza e della Brianza,Lombardia,163193000,Sistema ARREDAMENTO,grande
6,POLIFORM SPA,226920130,Como,Lombardia,161540108,Sistema ARREDAMENTO,grande
7,VENETA CUCINE SPA,3764840264,Treviso,Veneto,147732905,Sistema ARREDAMENTO,grande
8,IMAB GROUP SPA,111950416,Pesaro Urbino,Marche,145431089,Sistema ARREDAMENTO,grande
9,MOLTENI E C.- SPA,694950965,Monza e della Brianza,Lombardia,145417171,Sistema ARREDAMENTO,grande


### Selezione aziende Lombardia - Monza e della Brianza

In [9]:
az_lomb = data[data['regione']=="Lombardia"]
az_lomb_mb = az_lomb[az_lomb['provincia'] == "Monza e della Brianza"]

### Selezione aziende MB medie e grandi

In [10]:
az_lomb_mb_medie_grandi = az_lomb_mb[(az_lomb_mb['dimensione']== "media") | (az_lomb_mb['dimensione']== "grande")]
az_lomb_mb_medie_grandi

,azienda,p.iva,provincia,regione,fatturato,sistema,dimensione
5,POLTRONA FRAU SPA,5079060017,Monza e della Brianza,Lombardia,163193000,Sistema ARREDAMENTO,grande
9,MOLTENI E C.- SPA,694950965,Monza e della Brianza,Lombardia,145417171,Sistema ARREDAMENTO,grande
10,CHATEAU D AX SPA,682600960,Monza e della Brianza,Lombardia,144462851,Sistema ARREDAMENTO,grande
17,MINOTTI SPA,683370969,Monza e della Brianza,Lombardia,113894021,Sistema ARREDAMENTO,grande
20,CASSINA SPA,976180968,Monza e della Brianza,Lombardia,107060933,Sistema ARREDAMENTO,grande
31,FLEXFORM SPA,695310961,Monza e della Brianza,Lombardia,69679810,Sistema ARREDAMENTO,grande
48,BOFFI SPA (MB),2642250969,Monza e della Brianza,Lombardia,49277513,Sistema ARREDAMENTO,media
55,GIORGETTI SPA,695300962,Monza e della Brianza,Lombardia,43667553,Sistema ARREDAMENTO,media
67,BAXTER SRL,913080966,Monza e della Brianza,Lombardia,37981607,Sistema ARREDAMENTO,media
96,CAP DESIGN SPA,4311900965,Monza e della Brianza,Lombardia,27935275,Sistema ARREDAMENTO,media
